In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://pathumthani.moc.go.th/th/content/category/getarticles/id/114"
headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Host": "pathumthani.moc.go.th",
    "Origin": "https://pathumthani.moc.go.th",
    "Referer": "https://pathumthani.moc.go.th/th/content/category/index/id/114",
    "Sec-Ch-Ua": '"Google Chrome";v="129", "Not;A=Brand";v="8", "Chromium";v="129"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

articles_data = []

<Response [200]>

In [9]:
for page_index in range(1, 4):
    data = {
        'pageIndex': page_index 
    }
    
    response = requests.post(url, headers=headers, data=data)

    if response.status_code == 200:
        print(f"Request successful for page index: {page_index}:")
        data = response.json()
        html_content = data.get('html')
        soup = BeautifulSoup(html_content, 'html.parser')
        articles = soup.find_all('div', class_='inforow')
        
        for article in articles:
            link_tag = article.find('a', href=True)
            if link_tag:
                link = f"https://pathumthani.moc.go.th{link_tag['href']}"
            else:
                link = None
            
            date_tag = article.find('div', class_='image-date')
            if date_tag:
                date = date_tag.get_text(strip=True)
            else:
                date = None
            articles_data.append({'Link': link, 'Date': date})
    else:
        print(f"Request failed for page index {page_index} with status code: {response.status_code}")

Request successful for page index: 1:
Request successful for page index: 2:
Request successful for page index: 3:


In [11]:
df = pd.DataFrame(articles_data)
output_file = "articles_data.xlsx"
df.to_excel(output_file, index=False)